<a href="https://colab.research.google.com/github/Kashish-x1/mcq-generator/blob/main/mcq_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")
print("Setup complete!")

Setup complete!


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
text = "The capital of France is Paris."  # You can change this
doc = nlp(text)

# Extract entities
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")

Entity: France, Label: GPE
Entity: Paris, Label: GPE


In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
text = """
The Greek historian knew what he was talking about. The Nile River fed Egyptian civilization for hundreds of years.
The Longest River the Nile is 4,160 miles long—the world’s longest river. It begins near the equator in Africa and
flows north to the Mediterranean Sea. In the south the Nile churns with cataracts. A cataract is a waterfall. Near the
sea the Nile branches into a delta. A delta is an area near a river’s mouth where the water deposits fine soil called silt.
In the delta, the Nile divides into many streams. The river is called the upper Nile in the south and the lower Nile in the
north. For centuries, heavy rains in Ethiopia caused the Nile to flood every summer. The floods deposited rich soil along the
Nile’s shores. This soil was fertile, which means it was good for growing crops. Unlike the Tigris and Euphrates,
the Nile River flooded at the same time every year, so farmers could predict when to plant their crops.
"""
doc = nlp(text)

# Extract entities and sentences
entities = [(ent.text, ent.label_) for ent in doc.ents]
sentences = [sent.text.strip() for sent in doc.sents]
print("Entities:", entities)
print("Sentences:", sentences[:3])  # Show first few

Entities: [('Greek', 'NORP'), ('Nile River', 'LOC'), ('Egyptian', 'NORP'), ('hundreds of years', 'DATE'), ('The Longest River', 'LOC'), ('4,160 miles', 'QUANTITY'), ('Africa', 'LOC'), ('the Mediterranean Sea', 'LOC'), ('Nile', 'ORG'), ('Nile', 'ORG'), ('Nile', 'LOC'), ('Nile', 'LOC'), ('Nile', 'LOC'), ('centuries', 'DATE'), ('Ethiopia', 'GPE'), ('Nile', 'LOC'), ('Nile', 'ORG'), ('Tigris', 'ORG'), ('Euphrates', 'ORG'), ('the Nile River', 'LOC')]
Sentences: ['The Greek historian knew what he was talking about.', 'The Nile River fed Egyptian civilization for hundreds of years.', 'The Longest River the Nile is 4,160 miles long—the world’s longest river.']


In [5]:
import random

def extract_facts(doc):
    facts = []
    for sent in doc.sents:
        for ent in sent.ents:
            if ent.label_ in ["GPE", "LOC"]:  # Locations
                if "long" in sent.text.lower():
                    facts.append({"type": "length", "entity": ent.text, "value": "4,160 miles"})
                elif "begins" in sent.text.lower():
                    facts.append({"type": "source", "entity": ent.text, "value": "near the equator in Africa"})
                elif "flows" in sent.text.lower():
                    facts.append({"type": "destination", "entity": ent.text, "value": "Mediterranean Sea"})
            elif ent.label_ == "DATE" and "flood" in sent.text.lower():
                facts.append({"type": "flood_time", "entity": "Nile River", "value": ent.text})
    return facts

def generate_question(fact):
    if fact["type"] == "length":
        return f"How long is the {fact['entity']}?"
    elif fact["type"] == "source":
        return f"Where does the {fact['entity']} begin?"
    elif fact["type"] == "destination":
        return f"Where does the {fact['entity']} flow to?"
    elif fact["type"] == "flood_time":
        return f"When did the {fact['entity']} typically flood?"
    return None

# Test it
facts = extract_facts(doc)
for fact in facts[:2]:  # Limit to first 2 for now
    print(f"Fact: {fact}")
    print(f"Question: {generate_question(fact)}")

Fact: {'type': 'length', 'entity': 'The Longest River', 'value': '4,160 miles'}
Question: How long is the The Longest River?
Fact: {'type': 'source', 'entity': 'Africa', 'value': 'near the equator in Africa'}
Question: Where does the Africa begin?


In [6]:
distractors = {
    "length": ["3,000 miles", "5,000 miles", "2,500 miles"],
    "source": ["South America", "Asia", "Antarctica"],
    "destination": ["Red Sea", "Indian Ocean", "Atlantic Ocean"],
    "flood_time": ["winter", "spring", "autumn"]
}

def generate_mcq(fact):
    question = generate_question(fact)
    if not question:
        return None
    correct_answer = fact["value"]
    options = [correct_answer] + distractors[fact["type"]]
    random.shuffle(options)
    return {"question": question, "options": options, "answer": correct_answer}

# Test it
mcqs = [generate_mcq(fact) for fact in facts if generate_mcq(fact)]
for mcq in mcqs[:2]:  # Show first 2
    print(f"Q: {mcq['question']}")
    print(f"A) {mcq['options'][0]} B) {mcq['options'][1]} C) {mcq['options'][2]} D) {mcq['options'][3]}")
    print(f"Correct: {mcq['answer']}\n")

Q: How long is the The Longest River?
A) 4,160 miles B) 2,500 miles C) 3,000 miles D) 5,000 miles
Correct: 4,160 miles

Q: Where does the Africa begin?
A) South America B) Asia C) Antarctica D) near the equator in Africa
Correct: near the equator in Africa



In [7]:
def generate_multiple_mcqs(doc, num_questions=5):
    facts = extract_facts(doc)
    mcqs = [generate_mcq(fact) for fact in facts if generate_mcq(fact)]
    if len(mcqs) < num_questions:
        print(f"Only {len(mcqs)} questions generated due to limited facts.")
        return mcqs
    return random.sample(mcqs, num_questions)  # Pick 5 randomly

# Generate 5 MCQs
num_questions = 5
mcqs = generate_multiple_mcqs(doc, num_questions)
for i, mcq in enumerate(mcqs, 1):
    print(f"Q{i}: {mcq['question']}")
    print(f"A) {mcq['options'][0]} B) {mcq['options'][1]} C) {mcq['options'][2]} D) {mcq['options'][3]}")
    print(f"Correct: {mcq['answer']}\n")

Only 4 questions generated due to limited facts.
Q1: How long is the The Longest River?
A) 4,160 miles B) 3,000 miles C) 2,500 miles D) 5,000 miles
Correct: 4,160 miles

Q2: Where does the Africa begin?
A) Asia B) near the equator in Africa C) Antarctica D) South America
Correct: near the equator in Africa

Q3: Where does the the Mediterranean Sea begin?
A) near the equator in Africa B) Asia C) South America D) Antarctica
Correct: near the equator in Africa

Q4: When did the Nile River typically flood?
A) winter B) centuries C) spring D) autumn
Correct: centuries



In [8]:
!pip install spacy
!python -m spacy download en_core_web_sm

import spacy
import random

nlp = spacy.load("en_core_web_sm")

# Input text
text = """
The Greek historian knew what he was talking about. The Nile River fed Egyptian civilization for hundreds of years.
The Longest River the Nile is 4,160 miles long—the world’s longest river. It begins near the equator in Africa and
flows north to the Mediterranean Sea. In the south the Nile churns with cataracts. A cataract is a waterfall. Near the
sea the Nile branches into a delta. A delta is an area near a river’s mouth where the water deposits fine soil called silt.
In the delta, the Nile divides into many streams. The river is called the upper Nile in the south and the lower Nile in the
north. For centuries, heavy rains in Ethiopia caused the Nile to flood every summer. The floods deposited rich soil along the
Nile’s shores. This soil was fertile, which means it was good for growing crops. Unlike the Tigris and Euphrates,
the Nile River flooded at the same time every year, so farmers could predict when to plant their crops.
"""
doc = nlp(text)

# Define distractors for each fact type
distractors = {
    "length": ["3,000 miles", "5,000 miles", "2,500 miles"],
    "source": ["South America", "Asia", "Antarctica"],
    "destination": ["Red Sea", "Indian Ocean", "Atlantic Ocean"],
    "flood_time": ["winter", "spring", "autumn"],
    "cataract": ["lake", "mountain", "valley"],
    "delta": ["desert", "forest", "plateau"]
}

# Step 3: Extract facts with more rules
def extract_facts(doc):
    facts = []
    for sent in doc.sents:
        for ent in sent.ents:
            if ent.label_ in ["GPE", "LOC"]:
                if "long" in sent.text.lower():
                    facts.append({"type": "length", "entity": ent.text, "value": "4,160 miles"})
                elif "begins" in sent.text.lower():
                    facts.append({"type": "source", "entity": ent.text, "value": "near the equator in Africa"})
                elif "flows" in sent.text.lower():
                    facts.append({"type": "destination", "entity": ent.text, "value": "Mediterranean Sea"})
                elif "cataract" in sent.text.lower():
                    facts.append({"type": "cataract", "entity": ent.text, "value": "waterfall"})
                elif "delta" in sent.text.lower():
                    facts.append({"type": "delta", "entity": ent.text, "value": "an area near a river’s mouth"})
            elif ent.label_ == "DATE" and "flood" in sent.text.lower():
                facts.append({"type": "flood_time", "entity": "Nile River", "value": ent.text})
    return facts

# Step 3: Generate questions
def generate_question(fact):
    if fact["type"] == "length":
        return f"How long is the {fact['entity']}?"
    elif fact["type"] == "source":
        return f"Where does the {fact['entity']} begin?"
    elif fact["type"] == "destination":
        return f"Where does the {fact['entity']} flow to?"
    elif fact["type"] == "flood_time":
        return f"When did the {fact['entity']} typically flood?"
    elif fact["type"] == "cataract":
        return f"What is a cataract in relation to the {fact['entity']}?"
    elif fact["type"] == "delta":
        return f"What is a delta in relation to the {fact['entity']}?"
    return None

# Step 4: Generate a full MCQ
def generate_mcq(fact):
    question = generate_question(fact)
    if not question:
        return None
    correct_answer = fact["value"]
    options = [correct_answer] + distractors[fact["type"]]
    random.shuffle(options)
    return {"question": question, "options": options, "answer": correct_answer}

# Step 5: Generate 5 MCQs
def generate_multiple_mcqs(doc, num_questions=5):
    facts = extract_facts(doc)
    mcqs = [generate_mcq(fact) for fact in facts if generate_mcq(fact)]
    if len(mcqs) < num_questions:
        print(f"Generated {len(mcqs)} MCQs. Add more rules for {num_questions}.")
        return mcqs
    return random.sample(mcqs, num_questions)

# Run the generator
num_questions = 5
mcqs = generate_multiple_mcqs(doc, num_questions)
for i, mcq in enumerate(mcqs, 1):
    print(f"Q{i}: {mcq['question']}")
    print(f"A) {mcq['options'][0]} B) {mcq['options'][1]} C) {mcq['options'][2]} D) {mcq['options'][3]}")
    print(f"Correct: {mcq['answer']}\n")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Q1: When did the Nile River typically flood?
A) winter B) autumn C) centuries D) spring
Correct: centuries

Q2: Where does the Africa begin?
A) Antarctica B) South America C) near the equator in Africa D) Asia
Correct: near the equator in Africa

Q3: What is a delta in relation to the Nile?
A) desert B) an area near a river’s mouth C) plateau D) forest
Correct: an area near a river’s mouth

Q4: How long is the The Longest River?
A) 2,500 miles B) 4,160 miles C) 5,000 miles D) 3,000 miles
Correct: 4,160 miles

Q5: W

In [9]:
!pip install flask spacy PyPDF2
!python -m spacy download en_core_web_sm

from flask import Flask, request, render_template
import spacy
import PyPDF2
import random

app = Flask(__name__, template_folder='templates')  # Specify template folder
nlp = spacy.load("en_core_web_sm")

# Distractors for each fact type
distractors = {
    "length": ["3,000 miles", "5,000 miles", "2,500 miles"],
    "source": ["South America", "Asia", "Antarctica"],
    "destination": ["Red Sea", "Indian Ocean", "Atlantic Ocean"],
    "flood_time": ["winter", "spring", "autumn"],
    "cataract": ["lake", "mountain", "valley"],
    "delta": ["desert", "forest", "plateau"]
}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
def extract_text_from_pdf(file):
    pdf_reader = PyPDF2.PdfReader(file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() or ""  # Handle None from extract_text
    return text

def extract_facts(doc):
    facts = []
    for sent in doc.sents:
        for ent in sent.ents:
            if ent.label_ in ["GPE", "LOC"]:
                if "long" in sent.text.lower():
                    facts.append({"type": "length", "entity": ent.text, "value": "4,160 miles"})
                elif "begins" in sent.text.lower():
                    facts.append({"type": "source", "entity": ent.text, "value": "near the equator in Africa"})
                elif "flows" in sent.text.lower():
                    facts.append({"type": "destination", "entity": ent.text, "value": "Mediterranean Sea"})
                elif "cataract" in sent.text.lower():
                    facts.append({"type": "cataract", "entity": ent.text, "value": "waterfall"})
                elif "delta" in sent.text.lower():
                    facts.append({"type": "delta", "entity": ent.text, "value": "an area near a river’s mouth"})
            elif ent.label_ == "DATE" and "flood" in sent.text.lower():
                facts.append({"type": "flood_time", "entity": "Nile River", "value": ent.text})
    return facts

def generate_question(fact):
    if fact["type"] == "length":
        return f"How long is the {fact['entity']}?"
    elif fact["type"] == "source":
        return f"Where does the {fact['entity']} begin?"
    elif fact["type"] == "destination":
        return f"Where does the {fact['entity']} flow to?"
    elif fact["type"] == "flood_time":
        return f"When did the {fact['entity']} typically flood?"
    elif fact["type"] == "cataract":
        return f"What is a cataract in relation to the {fact['entity']}?"
    elif fact["type"] == "delta":
        return f"What is a delta in relation to the {fact['entity']}?"
    return None

def generate_mcq(fact):
    question = generate_question(fact)
    if not question:
        return None
    correct_answer = fact["value"]
    options = [correct_answer] + distractors[fact["type"]]
    random.shuffle(options)
    return {"question": question, "options": options, "answer": correct_answer}

def generate_multiple_mcqs(text, num_questions):
    doc = nlp(text)
    facts = extract_facts(doc)
    mcqs = [generate_mcq(fact) for fact in facts if generate_mcq(fact)]
    if not mcqs:
        return [{"question": "No facts found in the document.", "options": ["N/A"], "answer": "N/A"}]
    if len(mcqs) < num_questions:
        return mcqs + [{"question": f"Only {len(mcqs)} facts available.", "options": ["N/A"], "answer": "N/A"}] * (num_questions - len(mcqs))
    return random.sample(mcqs, num_questions)

In [11]:
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files or 'num_questions' not in request.form:
            return "Missing file or number of questions", 400
        file = request.files['file']
        num_questions = request.form['num_questions']

        if file.filename == '' or not num_questions.isdigit():
            return "Invalid file or number of questions", 400

        num_questions = int(num_questions)
        if num_questions <= 0:
            return "Number of questions must be positive", 400

        text = extract_text_from_pdf(file)
        mcqs = generate_multiple_mcqs(text, num_questions)
        return render_template('results.html', mcqs=mcqs)
    return render_template('upload.html')

In [12]:
!mkdir -p templates

In [13]:
with open('templates/upload.html', 'w') as f:
    f.write('''<!DOCTYPE html>
    <html>
    <head>
        <title>MCQ Generator</title>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap" rel="stylesheet">
        <style>
            body {
                background: linear-gradient(135deg, #74ebd5, #acb6e5);
                font-family: 'Poppins', sans-serif;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
            }
            .container {
                background: white;
                padding: 30px;
                border-radius: 15px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
                text-align: center;
                width: 400px;
            }
            h1 {
                color: #333;
                margin-bottom: 20px;
            }
            label {
                font-size: 18px;
                color: #555;
                display: block;
                margin: 10px 0 5px;
            }
            input[type="file"], input[type="number"] {
                width: 100%;
                padding: 10px;
                margin-bottom: 15px;
                border: 2px solid #74ebd5;
                border-radius: 5px;
                font-size: 16px;
            }
            input[type="submit"] {
                background-color: #74ebd5;
                color: white;
                padding: 12px 20px;
                border: none;
                border-radius: 5px;
                font-size: 18px;
                cursor: pointer;
                transition: background-color 0.3s;
            }
            input[type="submit"]:hover {
                background-color: #acb6e5;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Upload a Document to Generate MCQs</h1>
            <form method="POST" enctype="multipart/form-data">
                <label for="file">Choose a PDF file:</label>
                <input type="file" id="file" name="file" accept=".pdf">
                <label for="num_questions">Number of Questions:</label>
                <input type="number" id="num_questions" name="num_questions" min="1" value="5">
                <input type="submit" value="Generate MCQs">
            </form>
        </div>
    </body>
    </html>''')

In [14]:
with open('templates/results.html', 'w') as f:
    f.write('''<!DOCTYPE html>
    <html>
    <head>
        <title>Generated MCQs</title>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap" rel="stylesheet">
        <style>
            body {
                background: linear-gradient(135deg, #74ebd5, #acb6e5);
                font-family: 'Poppins', sans-serif;
                display: flex;
                justify-content: center;
                align-items: center;
                min-height: 100vh;
                margin: 0;
                padding: 20px;
            }
            .container {
                background: white;
                padding: 30px;
                border-radius: 15px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
                width: 600px;
                max-width: 100%;
            }
            h1 {
                color: #333;
                text-align: center;
                margin-bottom: 20px;
            }
            h3 {
                color: #74ebd5;
                margin: 20px 0 10px;
            }
            ul {
                list-style: none;
                padding: 0;
            }
            li {
                background: #f9f9f9;
                padding: 10px;
                margin: 5px 0;
                border-radius: 5px;
                transition: background 0.3s;
            }
            li:hover {
                background: #e0f7fa;
            }
            p {
                color: #555;
                font-weight: 600;
                margin: 15px 0;
            }
            a {
                display: inline-block;
                background-color: #acb6e5;
                color: white;
                padding: 10px 20px;
                text-decoration: none;
                border-radius: 5px;
                margin-top: 20px;
                transition: background-color 0.3s;
            }
            a:hover {
                background-color: #74ebd5;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Generated MCQs</h1>
            {% for mcq in mcqs %}
                <h3>Q{{ loop.index }}: {{ mcq.question }}</h3>
                <ul>
                    {% for option in mcq.options %}
                        <li>{{ loop.index0 | to_letter }}: {{ option }}</li>
                    {% endfor %}
                </ul>
                <p><strong>Correct Answer:</strong> {{ mcq.answer }}</p>
            {% endfor %}
            <a href="/">Back to Upload</a>
        </div>
    </body>
    </html>''')

In [ ]:
# Install required libraries
!pip install flask spacy PyPDF2 pyngrok

# Install spaCy model
!python -m spacy download en_core_web_sm

# Import libraries
from flask import Flask, request, render_template
import spacy
import PyPDF2
import random
from pyngrok import ngrok

# Set up Flask app
app = Flask(__name__, template_folder='templates')
nlp = spacy.load("en_core_web_sm")

# Custom filter to turn numbers into letters (0 -> A, 1 -> B, etc.)
def to_letter(index):
    return chr(65 + index)
app.jinja_env.filters['to_letter'] = to_letter

# Define distractors for MCQs
distractors = {
    "length": ["3,000 miles", "5,000 miles", "2,500 miles"],
    "source": ["South America", "Asia", "Antarctica"],
    "destination": ["Red Sea", "Indian Ocean", "Atlantic Ocean"],
    "flood_time": ["winter", "spring", "autumn"],
    "cataract": ["lake", "mountain", "valley"],
    "delta": ["desert", "forest", "plateau"]
}

# Function to extract text from a PDF
def extract_text_from_pdf(file):
    pdf_reader = PyPDF2.PdfReader(file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() or ""
    return text

# Function to find facts in the text
def extract_facts(doc):
    facts = []
    for sent in doc.sents:
        for ent in sent.ents:
            if ent.label_ in ["GPE", "LOC"]:
                if "long" in sent.text.lower():
                    facts.append({"type": "length", "entity": ent.text, "value": "4,160 miles"})
                elif "begins" in sent.text.lower():
                    facts.append({"type": "source", "entity": ent.text, "value": "near the equator in Africa"})
                elif "flows" in sent.text.lower():
                    facts.append({"type": "destination", "entity": ent.text, "value": "Mediterranean Sea"})
                elif "cataract" in sent.text.lower():
                    facts.append({"type": "cataract", "entity": ent.text, "value": "waterfall"})
                elif "delta" in sent.text.lower():
                    facts.append({"type": "delta", "entity": ent.text, "value": "an area near a river’s mouth"})
            elif ent.label_ == "DATE" and "flood" in sent.text.lower():
                facts.append({"type": "flood_time", "entity": "Nile River", "value": ent.text})
    return facts

# Function to make a question from a fact
def generate_question(fact):
    if fact["type"] == "length":
        return f"How long is the {fact['entity']}?"
    elif fact["type"] == "source":
        return f"Where does the {fact['entity']} begin?"
    elif fact["type"] == "destination":
        return f"Where does the {fact['entity']} flow to?"
    elif fact["type"] == "flood_time":
        return f"When did the {fact['entity']} typically flood?"
    elif fact["type"] == "cataract":
        return f"What is a cataract in relation to the {fact['entity']}?"
    elif fact["type"] == "delta":
        return f"What is a delta in relation to the {fact['entity']}?"
    return None

# Function to create an MCQ
def generate_mcq(fact):
    question = generate_question(fact)
    if not question:
        return None
    correct_answer = fact["value"]
    options = [correct_answer] + distractors[fact["type"]]
    random.shuffle(options)
    return {"question": question, "options": options, "answer": correct_answer}

# Function to generate MCQs without placeholders
def generate_multiple_mcqs(text, num_questions):
    doc = nlp(text)
    facts = extract_facts(doc)
    mcqs = [generate_mcq(fact) for fact in facts if generate_mcq(fact)]
    if not mcqs:
        return [{"question": "No facts found in the document.", "options": ["N/A"], "answer": "N/A"}]
    return mcqs[:num_questions] if mcqs else mcqs

# Define the main page of the website
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files or 'num_questions' not in request.form:
            return "Missing file or number of questions", 400
        file = request.files['file']
        num_questions = request.form['num_questions']

        if file.filename == '' or not num_questions.isdigit():
            return "Invalid file or number of questions", 400

        num_questions = int(num_questions)
        if num_questions <= 0:
            return "Number of questions must be positive", 400

        text = extract_text_from_pdf(file)
        mcqs = generate_multiple_mcqs(text, num_questions)
        return render_template('results.html', mcqs=mcqs)
    return render_template('upload.html')

# Create the templates folder and styled files
!mkdir -p templates
with open('templates/upload.html', 'w') as f:
    f.write('''<!DOCTYPE html>
    <html>
    <head>
        <title>MCQ Generator</title>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap" rel="stylesheet">
        <style>
            body {
                background: linear-gradient(135deg, #74ebd5, #acb6e5);
                font-family: 'Poppins', sans-serif;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
            }
            .container {
                background: white;
                padding: 30px;
                border-radius: 15px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
                text-align: center;
                width: 400px;
            }
            h1 {
                color: #333;
                margin-bottom: 20px;
            }
            label {
                font-size: 18px;
                color: #555;
                display: block;
                margin: 10px 0 5px;
            }
            input[type="file"], input[type="number"] {
                width: 100%;
                padding: 10px;
                margin-bottom: 15px;
                border: 2px solid #74ebd5;
                border-radius: 5px;
                font-size: 16px;
            }
            input[type="submit"] {
                background-color: #74ebd5;
                color: white;
                padding: 12px 20px;
                border: none;
                border-radius: 5px;
                font-size: 18px;
                cursor: pointer;
                transition: background-color 0.3s;
            }
            input[type="submit"]:hover {
                background-color: #acb6e5;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Upload a Document to Generate MCQs</h1>
            <form method="POST" enctype="multipart/form-data">
                <label for="file">Choose a PDF file:</label>
                <input type="file" id="file" name="file" accept=".pdf">
                <label for="num_questions">Number of Questions:</label>
                <input type="number" id="num_questions" name="num_questions" min="1" value="5">
                <input type="submit" value="Generate MCQs">
            </form>
        </div>
    </body>
    </html>''')

with open('templates/results.html', 'w') as f:
    f.write('''<!DOCTYPE html>
    <html>
    <head>
        <title>Generated MCQs</title>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap" rel="stylesheet">
        <style>
            body {
                background: linear-gradient(135deg, #74ebd5, #acb6e5);
                font-family: 'Poppins', sans-serif;
                display: flex;
                justify-content: center;
                align-items: center;
                min-height: 100vh;
                margin: 0;
                padding: 20px;
            }
            .container {
                background: white;
                padding: 30px;
                border-radius: 15px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
                width: 600px;
                max-width: 100%;
            }
            h1 {
                color: #333;
                text-align: center;
                margin-bottom: 20px;
            }
            h3 {
                color: #74ebd5;
                margin: 20px 0 10px;
            }
            ul {
                list-style: none;
                padding: 0;
            }
            li {
                background: #f9f9f9;
                padding: 10px;
                margin: 5px 0;
                border-radius: 5px;
                transition: background 0.3s;
            }
            li:hover {
                background: #e0f7fa;
            }
            p {
                color: #555;
                font-weight: 600;
                margin: 15px 0;
            }
            a {
                display: inline-block;
                background-color: #acb6e5;
                color: white;
                padding: 10px 20px;
                text-decoration: none;
                border-radius: 5px;
                margin-top: 20px;
                transition: background-color 0.3s;
            }
            a:hover {
                background-color: #74ebd5;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Generated MCQs</h1>
            {% for mcq in mcqs %}
                <h3>Q{{ loop.index }}: {{ mcq.question }}</h3>
                <ul>
                    {% for option in mcq.options %}
                        <li>{{ loop.index0 | to_letter }}: {{ option }}</li>
                    {% endfor %}
                </ul>
                <p><strong>Correct Answer:</strong> {{ mcq.answer }}</p>
            {% endfor %}
            <a href="/">Back to Upload</a>
        </div>
    </body>
    </html>''')

# Set up ngrok
ngrok.set_auth_token("2vArm8TgPBEeg6QrV9bMMJDRe0S_PYPr1jSt58dktfRf2nH6")  # Replace with your token
public_url = ngrok.connect(5000)
print(f"Access your website at: {public_url}")

# Start the Flask app
app.run(host='0.0.0.0', port=5000)

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Access your website at: NgrokTunnel: "https://cad8-104-199-136-146.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Apr/2025 11:09:33] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2025 11:09:34] "GET /favicon.ico HTTP/1.1" 404 -
